The main focus of this notebook are surface waves and their dispersion. 
Using synthetic seismograms generated with Instaseis (http://instaseis.net/), we try to recover the dispersion curves of Love and Rayleigh waves in the PREM model.

Author: Thomas Ulrich

In [ ]:
import instaseis
import obspy
from obspy.taup.taup_geo import calc_dist
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from mpl_toolkits.basemap import Basemap
import pycwt as wavelet
import pandas as pd

# db = instaseis.open_db("syngine://prem_a_10s")
db = instaseis.open_db("syngine://prem_i_2s")

We first generate synthetic seismograms with Instaseis by specifying moment tensor, source and receiver locations.

In [ ]:
# Source
lats = 89.91
lons = 74.4940
# receiver
latr = 42.6390
lonr = 74.4940

# Change to True to plot a map with station and receiver
if False:
    # Draw a map with receiver and source location
    lon_0 = 0.5 * (lons + lonr)
    lat_0 = 0.5 * (lats + latr)
    m = Basemap(projection="ortho", lon_0=lon_0, lat_0=lat_0, resolution="l")
    m.drawcoastlines()
    m.fillcontinents(color="lightgrey", lake_color="white")
    m.drawparallels(np.arange(-90.0, 120.0, 30.0))
    m.drawmeridians(np.arange(0.0, 420.0, 60.0))
    xi, yi = m(lons, lats)
    m.scatter(xi, yi, 200, color="r", marker="*", edgecolor="k", zorder=3)
    xi, yi = m(lonr, latr)
    m.scatter(xi, yi, 200, color="r", marker="v", edgecolor="k", zorder=3)
    plt.show()

In [ ]:
receiver = instaseis.Receiver(latitude=latr, longitude=lonr, network="AB", station="CED")
t0 = obspy.UTCDateTime(2011, 1, 2, 3, 4, 5)
source = instaseis.Source(
    latitude=lats,
    longitude=lons,
    depth_in_m=12000,
    m_rr=4.710000e24 / 1e7,
    m_tt=3.810000e22 / 1e7,
    m_pp=-4.740000e24 / 1e7,
    m_rt=3.990000e23 / 1e7,
    m_rp=-8.050000e23 / 1e7,
    m_tp=-1.230000e24 / 1e7,
    origin_time=t0,
)
st = db.get_seismograms(
    source=source, receiver=receiver, components=["Z", "R", "T"], kind="displacement"
)
# convert to micro_m
for i in range(3):
    st[i].data *= 1e6
st.plot()

## Group Velocity

If dispersion occurs, waves of different wavelengths travel at different speeds.  
In the following, we band-pass filter the seismograms around various central periods, ranging from 5 to 250s.

In [ ]:
myPeriods = [5.0, 7.5, 10, 12.5, 15, 20, 50, 100, 150, 200, 250]
nper = len(myPeriods)

figall, axarr = plt.subplots(nper + 1, 1, figsize=(14, 9), dpi=160, sharex=True, sharey=False)

iZRT = 1
ti = st[iZRT].times(reftime=t0)
axarr[0].plot(ti, st[iZRT].data, label="no filter")
axarr[0].legend(loc="upper left")

pickedArrival = np.zeros((nper,))
for i, T in enumerate(myPeriods):
    st_temp = st.copy()
    st_temp.filter("bandpass", freqmin=0.85 / T, freqmax=1.15 / T, corners=4, zerophase=True)
    axarr[i + 1].plot(
        ti,
        st_temp[iZRT].data,
        label=f"T={T:.1f}s",
    )
    # Envelope of filtered data
    data_envelope = obspy.signal.filter.envelope(st_temp[iZRT].data)
    axarr[i + 1].plot(ti, data_envelope.data)
    idmax = np.argmax(data_envelope.data)
    pickedArrival[i] = ti[idmax]
    axarr[i + 1].legend(loc="upper left")


axarr[0].set_xlim([0, 2200])

axarr[nper].set_xlabel("time (s)")
axarr[nper // 2].set_ylabel("displacement (um)")
plt.show()

d = {"filter_Tc": myPeriods, "picked_arrival": pickedArrival}
df = pd.DataFrame(data=d)
pd.options.display.float_format = "{:.0f}".format
print(df)

The idea of using a narrow filter to pick the arrival is imprecise and not applicable to real data. Instead, seismologist use a wavelet transform approach to track the central frequency with time ("A wavelet is a wave-like oscillation with an amplitude that begins at zero, increases or decreases, and then returns to zero one or more times", https://en.wikipedia.org/wiki/Wavelet). Such a solution is implemented below (inspired from http://regeirk.github.io/pycwt/tutorial.html):

In [ ]:
def compute_wavelet_power_spectrum(mydata, dt):
    # we normalize the data
    dat_norm = mydata / mydata.std()

    mother = wavelet.Morlet(6)

    s0 = 2 * dt  # Starting scale
    dj = 1 / 12  # Twelve sub-octaves per octaves
    J = 7 / dj  # Seven powers of two with dj sub-octaves

    # wave is shaped (nfreq, nti), with nfreq = J + 1 = 7 * 12 +1,
    # and nti the number of samples of the signal
    wave, scales, freqs, coi, fft, fftfreqs = wavelet.cwt(dat_norm, dt, dj, s0, J, mother)
    iwave = wavelet.icwt(wave, scales, dt, dj, mother) * std

    period = 1 / freqs

    # compute power spectrum
    power = (np.abs(wave)) ** 2
    # rectify the power spectrum according to the suggestions proposed by Liu et al. (2007)
    power /= scales[:, None]
    return period, power, coi

dt = st[iZRT].stats.delta

period, power, coi = compute_wavelet_power_spectrum(st[iZRT].data, dt)

# Plot wavelet power spectrum figure
figall, ax = plt.subplots()
levels = [16 / 2**i for i in range(10, -1, -1)]
ax.contourf(
    ti, np.log2(period), np.log2(power), np.log2(levels), extend="both", cmap=plt.cm.viridis
)

if False:
    # change to True to also plot the "cone of influence" as a hatched area
    ax.fill(np.concatenate([ti, ti[-1:] + dt, ti[-1:] + dt,
                               ti[:1] - dt, ti[:1] - dt]),
            np.concatenate([np.log2(coi), [1e-9], np.log2(period[-1:]),
                               np.log2(period[-1:]), [1e-9]]),
            'k', alpha=0.3, hatch='x')


ax.set_title(f"Wavelet Power Spectrum ({mother.name})")
ax.set_xlabel("time (s)")
ax.set_ylabel("period (s)")
ax.grid()
Yticks = 2 ** np.arange(np.ceil(np.log2(period.min())), np.ceil(np.log2(period.max())))

ax.set_yticks(np.log2(Yticks))
ax.set_yticklabels(Yticks)

plt.xlim(0, 2200)

In [ ]:
def pickArrivalWavelet(ti, power, period, tmin=1050):
    # pick arrival time from the wavelet Power Spectrum data
    maxs = np.amax(power, axis=1)
    # enforce that the arrival time should be greater than tmin
    idmin = np.argmin(np.abs(ti - tmin))
    idmaxs = np.argmax(power[:, idmin:], axis=1) + idmin

    pickedArrivalWavelet = np.zeros_like(period)
    for i in range(power.shape[0]):
        pickedArrivalWavelet[i] = ti[idmaxs[i]]
    return pickedArrivalWavelet

pickedArrivalWavelet = pickArrivalWavelet(ti, power, period)
plt.plot(period, pickedArrivalWavelet)
plt.xlabel("period (s)")
plt.ylabel("picked time (s)")
plt.show()

Note that for period in the range 30 to 60s, there is energy arriving very early (between 500s and 1000s following the earthquake onset). We need to manually enforce that the picked time is greater than 1050s, to avoid picking wrong arrivals.   
Next, we compute the distance between source and receivers using obspy.

In [ ]:
dist_degree_sr = calc_dist(
    source_latitude_in_deg=lats,
    source_longitude_in_deg=lons,
    receiver_latitude_in_deg=latr,
    receiver_longitude_in_deg=lonr,
    radius_of_planet_in_km=6371,
    flattening_of_planet=0,
)
dist_km_sr = 6371 * dist_degree_sr * 2 * np.pi / 360.0
print(f"distance source-receivers: {dist_km_sr:.1f} km ({dist_degree_sr:.1f} deg)")

Now we plot the inferred dispersion curves and compare them with theoretical estimates.

In [ ]:
plt.plot(myPeriods, dist_km_sr / pickedArrival, "x", label="this notebook")
plt.plot(period, dist_km_sr / pickedArrivalWavelet, "o", label="this notebook, wavelet")
plt.xlabel("period (s)")
plt.ylabel("group velocity (km/s)")


# source: http://online.kitp.ucsb.edu/online/earth_m06/masters2/oh/08.html
PREM_groupVelRayleigh = """19.54918	3.352409
22.592722	3.5495863
29.88685	3.7826443
48.336567	3.9184444
70.88198	3.88144
108.34908	3.7869449
163.06877	3.6745753
207.68121	3.600516
245.99522	3.5769787
271.7795	3.611183
298.8952	3.7124586
"""
# source: http://online.kitp.ucsb.edu/online/earth_m06/masters2/oh/08.html
PREM_groupVelLove = """
19.49333	3.26055
23.80026	3.4672015
28.953934	3.7215185
37.50143	3.9565687
44.815434	4.1042824
60.388584	4.1978517
83.67851	4.264917
122.87113	4.283009
160.17804	4.28892
217.57329	4.2980137
264.88776	4.321354
299.29742	4.3386884
"""

datastring = PREM_groupVelLove if iZRT==2 else PREM_groupVelRayleigh
a = np.fromstring(datastring, sep="	")

PREM_groupVel = a.reshape((len(a) // 2, 2))
plt.plot(PREM_groupVel[:, 0], PREM_groupVel[:, 1], label="PREM (G. Laske)")
plt.legend()
plt.show()

## Phase velocity

The frequency-dependent phase of a seismogram can be obtained by taking the Fourrier transform of it. It can be written as follows: 

\begin{equation*}
\Phi(\omega) = \omega t - k(\omega) x + \Phi_i(\omega) + 2n\pi = \omega t - \omega x / c(\omega)+ \Phi_i(\omega) + 2n\pi
\end{equation*}

Where $\omega t - \omega x / c(\omega)$ is the phase due to the propagation in space and time,  
$\Phi_i(\omega)$ depends on the source and  
$2n\pi$ reflects the periodicity of the phase.  
To compute the phase velocity we need to track the same wave at two receivers.  
Assuming that the receivers are at distance $x_1$ and $x_2$ from the source, and the wave passes at time $t_1$ and $t_2$ at each receivers, then the phase of both receivers would be:  

\begin{equation*}
\Phi_1(\omega) =   \omega t_1 - \omega x_1 / c(\omega)+ \Phi_i(\omega) + 2n\pi
\end{equation*}
and  
\begin{equation*}
\Phi_2(\omega) = \omega t_2 - \omega x_2 / c(\omega)+ \Phi_i(\omega) + 2m\pi
\end{equation*}

Then c can be obtained from the difference $\Phi_{21}(\omega)$ between $\Phi_2(\omega)$ and $\Phi_1(\omega)$:
\begin{equation*}
c(\omega) = \omega (x_2-x_1)/(\omega (t_2-t_1)+ 2(m-n)\pi- \Phi_{21}(\omega))
\end{equation*}
the term m-n is found empirically ensuring that the phase velocity is reasonable for long periods.  
source: Stein and Wysession.

In [ ]:
receiver2 = instaseis.Receiver(latitude=latr - 6, longitude=lonr)
st2 = db.get_seismograms(
    source=source, receiver=receiver2, components=["Z", "R", "T"], kind="displacement"
)
# convert to micro_m
for i in range(3):
    st2[i].data *= 1e6
st.plot()
st2.plot()

In [ ]:
myPeriods = [50, 100, 150, 200, 250]
nper = len(myPeriods)

figall, axarr = plt.subplots(nper + 1, 1, figsize=(14, 9), dpi=160, sharex=True, sharey=False)

axarr[0].plot(ti, st[iZRT].data, label="no filter, r1")
axarr[0].plot(ti, st2[iZRT].data, label="no filter, r2")
axarr[0].legend()

pickedArrival = np.zeros((nper, 3))
pickedArrival[:, 0] = myPeriods

for i, T in enumerate(myPeriods):
    for k, sti in enumerate([st, st2]):
        st_temp = sti.copy()
        st_temp.filter("bandpass", freqmin=0.85 / T, freqmax=1.15 / T, corners=4, zerophase=True)
        axarr[i + 1].plot(
            ti,
            st_temp[iZRT].data,
            label=f"T={T:.1f}s",
        )
        # Envelope of filtered data
        data_envelope = obspy.signal.filter.envelope(st_temp[iZRT].data)
        axarr[i + 1].plot(ti, data_envelope.data)
        idmax = np.argmax(data_envelope.data)
        pickedArrival[i, k + 1] = st_temp[iZRT].times(reftime=t0)[idmax]
        axarr[i + 1].legend()
print(pickedArrival)

axarr[5].set_xlabel("time (s)")
axarr[5].set_ylabel("displacement (um)")
plt.show()